In [8]:
import datetime
now = datetime.datetime.now()
month = now.month
month

if 2 != 3:
    print("good")

good


In [2]:
#選擇權下單範例

# 步驟
# 取得帳戶資訊
# 確立合約
# 確立order
# 送出order
 
# globals.position = globals.api.list_positions(globals.api.futopt_account)
# Offset.CLOSE: A closing offset (CO) order is a limit order that is executed at market close but can be placed anytime during the trading day. 

import shioaji as sj
from shioaji.constant import Action, OptionRight, StockPriceType, FuturesOrderType, FuturesOCType #選擇權下單，多匯入一個OptionRight常數

api = sj.Shioaji()
api.login(
    person_id='R124743829', 
    passwd='366375Dauhui'
)

#contract = globals.api.Contracts.Options[option_code] #選擇權Contract
contract = api.Contracts.Options['TXO14550L2'] #選擇權Contract
contract_snap = api.snapshots([contract])
print(contract_snap)
order = api.Order(
    action=Action.Buy,
    price=170, #價格
    quantity=2, #口數
    price_type=StockPriceType.LMT, #MKT: 市價 LMT: 限價
    order_type=FuturesOrderType.ROD,
    octype=FuturesOCType.Cover, #倉別，收盤時平倉
    OptionRight=OptionRight.Call, #選擇權類型
    account=api.futopt_account #下單帳戶指定期貨帳戶
)

#trade = api.place_order(contract, order)
#print(type(trade))


snapshots = api.snapshots([api.Contracts.Options.TXO.TXO20221214550C])
snapshots

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
[Snapshot(ts=1670545193486000000, code='TXO14550L2', exchange='TAIFEX', open=200.0, high=266.0, low=200.0, close=248.0, tick_type=<TickType.Sell: 'Sell'>, change_price=48.0, change_rate=24.0, change_type=<ChangeType.Up: 'Up'>, average_price=229.02, volume=2, total_volume=131, amount=496, total_amount=30002, yesterday_volume=127.0, buy_price=251.0, buy_volume=20.0, sell_price=255.0, sell_volume=9, volume_ratio=1.03)]


[Snapshot(ts=1670545193486000000, code='TXO14550L2', exchange='TAIFEX', open=200.0, high=266.0, low=200.0, close=248.0, tick_type=<TickType.Sell: 'Sell'>, change_price=48.0, change_rate=24.0, change_type=<ChangeType.Up: 'Up'>, average_price=229.02, volume=2, total_volume=131, amount=496, total_amount=30002, yesterday_volume=127.0, buy_price=251.0, buy_volume=20.0, sell_price=255.0, sell_volume=9, volume_ratio=1.03)]

In [ ]:
option_code = globals.at_the_money_code +  snap.get_option_code(globals.simulation_optionright)
print(option_code)
globals.contract = positions.fill_contract(option_code)

# Subscribe to the close price of the contract
# contract will be ordered in third BidAsk
# Subscribe to the close price of the contract
globals.api.quote.subscribe(
    globals.contract,
    quote_type = sj.constant.QuoteType.BidAsk, # or 'bidask'
    version = sj.constant.QuoteVersion.v1, # or 'v1'
)

@globals.api.on_bidask_fop_v1()
def quote_callback(exchange:sj.Exchange, bidask:BidAskFOPv1):
    #always avoid making calulations inside the callback function.
    """
    Quoting subscribe function. It is called every tick(theoretically)

    :global param: contract (str)
    :global param: third_best_buy_price (int)
    :global param: third_best_sell_price (int)
    :return: None
    """
    print("ask_price: ", bidask['ask_price'], "bid_price: ", bidask['bid_price'])

    # 取第三檔掛單價格，是為了避免夜盤時價差太大可能造成的風險
    globals.third_best_buy_price = bidask['ask_price'][2] # 第三檔之最佳買價
    globals.third_best_sell_price = bidask['bid_price'][2] # 第三檔之最佳買賣價